In [1]:
import pandas as pd
import fractions
import re
import string
import nltk
import spacy 
import numpy as np
import math
from tqdm import tqdm 

from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy import displacy
pd.set_option('display.max_colwidth',200)


data=pd.read_csv("amazon_baby.csv",error_bad_lines=False)
data

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion not worth keeping. I also ordered someImse Vimse Cloth Wipes-Ocean Blue-12 countwhich are larger, had a nicer, softer texture and just seemed higher ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it looks...fit the full size bed perfectly...would recommend to anyone looking for this type of quilt,5
3,Stop Pacifier Sucking without tears with Thumbuddy To Love's Binky Fairy Puppet and Adorable Book,"This is a product well worth the purchase. I have not found anything else like this, and it is a positive, ingenious approach to losing the binky. What I love most about this product is how much...",5
4,Stop Pacifier Sucking without tears with Thumbuddy To Love's Binky Fairy Puppet and Adorable Book,"All of my kids have cried non-stop when I tried to ween them off their pacifier, until I found Thumbuddy To Love's Binky Fairy Puppet. It is an easy way to work with your kids to allow them to un...",5
...,...,...,...
183526,Baby Teething Necklace for Mom Pretty Donut Shaped Pendant (Smokey Black),"Such a great idea! very handy to have and look really cool too, not to mention great for baby's gums to help reduce the pain of teething!",5
183527,Baby Teething Necklace for Mom Pretty Donut Shaped Pendant (Smokey Black),This product rocks! It is a great blend of function and fashion. Aside from the unique concept of having something easily accessible and cool for baby to play and soothe themselves as cool acces...,5
183528,Abstract 2 PK Baby / Toddler Training Cup (Pink),This item looks great and cool for my kids....I know this Company and its great in quality...,5
183529,"Baby Food Freezer Tray - Bacteria Resistant, BPA and Phthalate Free - FREE Recipe E-book - Silicone Storage Container with Lid for Homemade Baby Food - 9 Easy-Out Portions - Lifetime Guarantee - P...",I am extremely happy with this product. I have been searching for a product like this for a long time and just not found the right one. I am a mum who is very into my organic products and unllike ...,5


In [2]:
nlp=spacy.load('en_core_web_sm')


In [3]:
data_text=data[["review"]]
data_text['index']=data_text.index

documents=data_text
documents

<ipython-input-3-586910c446a7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_text['index']=data_text.index


,review,index
0,"These flannel wipes are OK, but in my opinion not worth keeping. I also ordered someImse Vimse Cloth Wipes-Ocean Blue-12 countwhich are larger, had a nicer, softer texture and just seemed higher ...",0
1,it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.,1
2,Very soft and comfortable and warmer than it looks...fit the full size bed perfectly...would recommend to anyone looking for this type of quilt,2
3,"This is a product well worth the purchase. I have not found anything else like this, and it is a positive, ingenious approach to losing the binky. What I love most about this product is how much...",3
4,"All of my kids have cried non-stop when I tried to ween them off their pacifier, until I found Thumbuddy To Love's Binky Fairy Puppet. It is an easy way to work with your kids to allow them to un...",4
...,...,...
183526,"Such a great idea! very handy to have and look really cool too, not to mention great for baby's gums to help reduce the pain of teething!",183526
183527,This product rocks! It is a great blend of function and fashion. Aside from the unique concept of having something easily accessible and cool for baby to play and soothe themselves as cool acces...,183527
183528,This item looks great and cool for my kids....I know this Company and its great in quality...,183528
183529,I am extremely happy with this product. I have been searching for a product like this for a long time and just not found the right one. I am a mum who is very into my organic products and unllike ...,183529


In [4]:
used_doc=documents[:5700]

In [5]:
used_doc

,review,index
0,"These flannel wipes are OK, but in my opinion not worth keeping. I also ordered someImse Vimse Cloth Wipes-Ocean Blue-12 countwhich are larger, had a nicer, softer texture and just seemed higher ...",0
1,it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.,1
2,Very soft and comfortable and warmer than it looks...fit the full size bed perfectly...would recommend to anyone looking for this type of quilt,2
3,"This is a product well worth the purchase. I have not found anything else like this, and it is a positive, ingenious approach to losing the binky. What I love most about this product is how much...",3
4,"All of my kids have cried non-stop when I tried to ween them off their pacifier, until I found Thumbuddy To Love's Binky Fairy Puppet. It is an easy way to work with your kids to allow them to un...",4
...,...,...
5695,This is a for sure MUST for babies! So much better then those hard plastic tubs.. and a lot easier then folding up towel in the sink! It stays warm like the bath water so your child doesnt get chi...,5695
5696,mildews so bad! i bought a second one thinking that I was doing something wrong. I rinsed and even put outside every night to dry completely. I now need to buy a bath for my 6 mo old since she can...,5696
5697,"The reason for the 1 star is because baby is not due until Dec 1 2013, and it has not been used. Mother to be is excited about it, but we won't know the thoughts until its used, then I'll re-rate it",5697
5698,This bathing pillow is the best! It keeps baby warm and is very comfy. My baby loves it - she can still kick around in the water while her head is elevated. I highly recommend this product for ...,5698


In [6]:
used_doc.dropna(subset = ["review"], inplace=True) # drop those rows which have NaN value cells

<ipython-input-6-ada661c3bb41>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  used_doc.dropna(subset = ["review"], inplace=True) # drop those rows which have NaN value cells


In [7]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
import nltk.stem as stemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/deth/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
doc_sample=used_doc[used_doc['index']==4].values[0][0]


In [9]:
test_text=nlp(doc_sample)



In [10]:
noun=[]
for tok in test_text:
    if tok.pos_=='NOUN':
        print(tok.text,'->',tok.pos_)
        noun.append(tok.text)

kids -> NOUN
stop -> NOUN
pacifier -> NOUN
way -> NOUN
kids -> NOUN
pacifier -> NOUN
part -> NOUN
book -> NOUN
gift -> NOUN
parents -> NOUN
soo -> NOUN
headaches -> NOUN
Thanks -> NOUN
book -> NOUN
rock -> NOUN


In [11]:
word_freq = []
for w in noun:
    word_freq.append(noun.count(w))

test_list=list(zip(noun, word_freq))
print("Pairs\n" + str(test_list))

Pairs
[('kids', 2), ('stop', 1), ('pacifier', 2), ('way', 1), ('kids', 2), ('pacifier', 2), ('part', 1), ('book', 2), ('gift', 1), ('parents', 1), ('soo', 1), ('headaches', 1), ('Thanks', 1), ('book', 2), ('rock', 1)]


In [12]:
for i in test_list:
    if i[1]==max(word_freq):
        print(i[0],i[1])

kids 2
pacifier 2
kids 2
pacifier 2
book 2
book 2


In [13]:
noun

['kids',
 'stop',
 'pacifier',
 'way',
 'kids',
 'pacifier',
 'part',
 'book',
 'gift',
 'parents',
 'soo',
 'headaches',
 'Thanks',
 'book',
 'rock']

In [14]:
stemmer=PorterStemmer()
def lemmatize_stemming(text):
      return stemmer.stem(WordNetLemmatizer().lemmatize(text,pos='v'))

def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) >3:
              result.append(lemmatize_stemming(token))
  
    return result

def preprocess1(text):
    test_text=nlp(text)
    noun=[]
    for tok in test_text:
        if tok.pos_=='NOUN':
#             print(tok.text,'->',tok.pos_)
            noun.append(tok.text)
    return noun

    



In [15]:
processed_docs1 = used_doc['review'].map(preprocess1)
processed_docs1[:10]

0            [flannel, wipes, opinion, countwhich, texture, quality, cloth, wipes, hands, faces, usingThirsties, months, issues, while, handles]
1                                                                                            [planet, bags, wipe, holder, osocozy, wipes, moist]
2                                                                                                                       [size, bed, type, quilt]
3                         [product, purchase, approach, binky, product, ownership, daughter, binky, fairy, artwork, chart, back, approach, tool]
4                                     [kids, stop, pacifier, way, kids, pacifier, part, book, gift, parents, soo, headaches, Thanks, book, rock]
5    [house, gift, book, pacifier, book, job, child, loss, item, doll, lots, movies, daughter, product, parent, pacifier, thumb, sucking, habit]
6                                                                                                      [book, alot, photos, cards,

In [16]:
dictionary=gensim.corpora.Dictionary(processed_docs1)
count=0

for k,v in dictionary.iteritems():
    print(k,v)
    count+=1
    if count>10:
        break

0 cloth
1 countwhich
2 faces
3 flannel
4 handles
5 hands
6 issues
7 months
8 opinion
9 quality
10 texture


In [ ]:
print(dictionary)


In [17]:
dictionary.filter_extremes(no_below=15,no_above=0.5)

In [18]:
bow_corpus=[dictionary.doc2bow(doc) for doc in processed_docs1]
bow_corpus[4]

[(20, 1), (21, 2), (22, 1), (23, 2), (24, 2), (25, 1), (26, 1), (27, 1)]

In [20]:
bow_doc_4=bow_corpus[4]
for i in range(len(bow_doc_4)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4[i][0], 
                                               dictionary[bow_doc_4[i][0]]),
bow_doc_4[i][1]))

Word 20 ("Thanks") appears 1 time.
Word 21 ("book") appears 2 time.
Word 22 ("gift") appears 1 time.
Word 23 ("kids") appears 2 time.
Word 24 ("pacifier") appears 2 time.
Word 25 ("parents") appears 1 time.
Word 26 ("part") appears 1 time.
Word 27 ("way") appears 1 time.


In [21]:
from gensim import corpora, models
tfidf=models.TfidfModel(bow_corpus)
corpus_tfidf=tfidf[bow_corpus]

from pprint import pprint

for doc in corpus_tfidf:
  pprint(doc)
  break

[(0, 0.2901732745443343),
 (1, 0.3572577297886199),
 (2, 0.21248377678976674),
 (3, 0.2716193176756163),
 (4, 0.12272001347251357),
 (5, 0.3307631085174361),
 (6, 0.22310337734314678),
 (7, 0.2666720213422817),
 (8, 0.6510656314854755)]


In [22]:
lda_model=gensim.models.LdaMulticore(bow_corpus,num_topics=10,id2word=dictionary,passes=2,workers=2)

In [36]:
for idx, topic in lda_model.print_topics(4):
      print('Topic:{}\nwords:{}'.format(idx,topic))

Topic:0
words:0.049*"gate" + 0.033*"baby" + 0.026*"product" + 0.025*"son" + 0.024*"seat" + 0.015*"carrier" + 0.014*"way" + 0.013*"sheet" + 0.013*"top" + 0.012*"head"
Topic:4
words:0.081*"gate" + 0.048*"tub" + 0.031*"baby" + 0.019*"product" + 0.019*"time" + 0.016*"door" + 0.016*"son" + 0.016*"one" + 0.013*"year" + 0.012*"months"
Topic:7
words:0.026*"stool" + 0.024*"daughter" + 0.020*"son" + 0.020*"bed" + 0.017*"product" + 0.015*"crib" + 0.014*"play" + 0.014*"seat" + 0.013*"kids" + 0.012*"toddler"
Topic:6
words:0.053*"baby" + 0.026*"one" + 0.026*"diaper" + 0.023*"product" + 0.022*"diapers" + 0.019*"months" + 0.015*"bags" + 0.014*"weeks" + 0.013*"bag" + 0.010*"pail"


In [40]:
lda_model_tfidf=gensim.models.LdaMulticore(corpus_tfidf,num_topics=10,id2word=dictionary,passes=2,workers=4)
for idx, topic in lda_model_tfidf.print_topics(4):
      print('Topic: {} Word: {}'.format(idx,topic))

Topic: 6 Word: 0.019*"potty" + 0.017*"seat" + 0.012*"chair" + 0.011*"stool" + 0.011*"son" + 0.011*"size" + 0.009*"diaper" + 0.009*"tub" + 0.009*"baby" + 0.009*"year"
Topic: 5 Word: 0.021*"bottles" + 0.017*"bottle" + 0.015*"baby" + 0.011*"nipples" + 0.010*"crib" + 0.010*"product" + 0.009*"problem" + 0.008*"pump" + 0.008*"problems" + 0.008*"sheet"
Topic: 3 Word: 0.032*"toy" + 0.015*"music" + 0.013*"son" + 0.011*"months" + 0.011*"baby" + 0.011*"product" + 0.011*"daughter" + 0.010*"month" + 0.009*"book" + 0.009*"thing"
Topic: 8 Word: 0.023*"product" + 0.021*"baby" + 0.016*"bottles" + 0.013*"bottle" + 0.012*"tub" + 0.011*"water" + 0.011*"gift" + 0.010*"time" + 0.009*"minutes" + 0.009*"gate"


In [42]:
# Performance evaluation by classifying sample document using LDA Bag of Words model
for index, score in sorted(lda_model[bow_corpus[4]],key=lambda tup:-1*tup[1]):
  print("\nScore:{}\t \nTopic:{}".format(score,lda_model.print_topic(index,10)))


Score:0.5447636842727661	 
Topic:0.026*"stool" + 0.024*"daughter" + 0.020*"son" + 0.020*"bed" + 0.017*"product" + 0.015*"crib" + 0.014*"play" + 0.014*"seat" + 0.013*"kids" + 0.012*"toddler"

Score:0.38854557275772095	 
Topic:0.049*"gate" + 0.033*"baby" + 0.026*"product" + 0.025*"son" + 0.024*"seat" + 0.015*"carrier" + 0.014*"way" + 0.013*"sheet" + 0.013*"top" + 0.012*"head"


In [43]:
# Performance evaluation by classifying sample document using LDA TF-IDF model.
for index, score in sorted(lda_model_tfidf[bow_corpus[4]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.4853854179382324	 
Topic: 0.019*"potty" + 0.017*"seat" + 0.012*"chair" + 0.011*"stool" + 0.011*"son" + 0.011*"size" + 0.009*"diaper" + 0.009*"tub" + 0.009*"baby" + 0.009*"year"

Score: 0.4479242265224457	 
Topic: 0.025*"tub" + 0.018*"bottles" + 0.013*"son" + 0.012*"seat" + 0.012*"potty" + 0.011*"baby" + 0.011*"daughter" + 0.011*"time" + 0.010*"toilet" + 0.010*"bottle"


In [44]:
unseen_document = 'This pacifier has always been an amazing product'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.5498700141906738	 Topic: 0.049*"gate" + 0.033*"baby" + 0.026*"product" + 0.025*"son" + 0.024*"seat"
Score: 0.050024889409542084	 Topic: 0.053*"baby" + 0.026*"one" + 0.026*"diaper" + 0.023*"product" + 0.022*"diapers"
Score: 0.05001718923449516	 Topic: 0.081*"gate" + 0.048*"tub" + 0.031*"baby" + 0.019*"product" + 0.019*"time"
Score: 0.05001521483063698	 Topic: 0.098*"bottles" + 0.035*"baby" + 0.024*"time" + 0.023*"nipples" + 0.021*"bottle"
Score: 0.05001511797308922	 Topic: 0.026*"stool" + 0.024*"daughter" + 0.020*"son" + 0.020*"bed" + 0.017*"product"
Score: 0.050014678388834	 Topic: 0.052*"toy" + 0.045*"son" + 0.028*"cups" + 0.018*"months" + 0.017*"baby"
Score: 0.05001425743103027	 Topic: 0.056*"baby" + 0.026*"bottles" + 0.022*"monitor" + 0.020*"bottle" + 0.020*"time"
Score: 0.05001157894730568	 Topic: 0.054*"baby" + 0.027*"months" + 0.025*"son" + 0.024*"diaper" + 0.024*"time"
Score: 0.050009746104478836	 Topic: 0.093*"bottle" + 0.078*"bottles" + 0.027*"baby" + 0.026*"nipples" 

In [46]:
unseen_document = 'That Pacifier is a very bad products'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.5499119758605957	 Topic: 0.052*"toy" + 0.045*"son" + 0.028*"cups" + 0.018*"months" + 0.017*"baby"
Score: 0.050017133355140686	 Topic: 0.098*"bottles" + 0.035*"baby" + 0.024*"time" + 0.023*"nipples" + 0.021*"bottle"
Score: 0.050012409687042236	 Topic: 0.054*"baby" + 0.027*"months" + 0.025*"son" + 0.024*"diaper" + 0.024*"time"
Score: 0.050009965896606445	 Topic: 0.056*"baby" + 0.026*"bottles" + 0.022*"monitor" + 0.020*"bottle" + 0.020*"time"
Score: 0.05000922828912735	 Topic: 0.026*"stool" + 0.024*"daughter" + 0.020*"son" + 0.020*"bed" + 0.017*"product"
Score: 0.05000898987054825	 Topic: 0.081*"gate" + 0.048*"tub" + 0.031*"baby" + 0.019*"product" + 0.019*"time"
Score: 0.05000854656100273	 Topic: 0.052*"baby" + 0.038*"daughter" + 0.035*"time" + 0.034*"seat" + 0.024*"bottles"
Score: 0.05000774562358856	 Topic: 0.053*"baby" + 0.026*"one" + 0.026*"diaper" + 0.023*"product" + 0.022*"diapers"
Score: 0.05000704899430275	 Topic: 0.093*"bottle" + 0.078*"bottles" + 0.027*"baby" + 0.026*"n

In [47]:
from gensim.test.utils import common_dictionary, common_corpus
from gensim.models import LsiModel

lsi_model=LsiModel(bow_corpus,num_topics=10,id2word=dictionary)

In [49]:
for idx, topic in lsi_model.print_topics(4):
      print('Topic:{}\nwords:{}'.format(idx,topic))

Topic:0
words:-0.513*"bottles" + -0.473*"baby" + -0.327*"bottle" + -0.232*"time" + -0.198*"tub" + -0.161*"son" + -0.150*"water" + -0.143*"months" + -0.127*"nipples" + -0.125*"product"
Topic:1
words:-0.553*"bottles" + 0.540*"tub" + 0.299*"baby" + -0.294*"bottle" + 0.133*"gate" + 0.125*"months" + 0.121*"bath" + 0.118*"seat" + 0.109*"son" + -0.106*"nipples"
Topic:2
words:-0.674*"gate" + 0.539*"tub" + -0.168*"seat" + 0.129*"water" + -0.117*"diaper" + -0.110*"product" + -0.110*"potty" + 0.103*"bottles" + 0.099*"bath" + -0.098*"one"
Topic:3
words:0.628*"gate" + -0.418*"seat" + 0.330*"tub" + -0.242*"potty" + -0.180*"diaper" + -0.146*"son" + -0.146*"toilet" + -0.119*"product" + 0.113*"bottles" + -0.104*"bag"


In [50]:
lsi_model_tfidf=LsiModel(corpus_tfidf,num_topics=10,id2word=dictionary)
for idx, topic in lsi_model_tfidf.print_topics(-1):
      print('Topic: {} Word: {}'.format(idx,topic))

Topic: 0 Word: 0.409*"bottles" + 0.302*"tub" + 0.270*"bottle" + 0.243*"baby" + 0.186*"son" + 0.167*"time" + 0.157*"water" + 0.154*"product" + 0.150*"months" + 0.147*"daughter"
Topic: 1 Word: -0.620*"tub" + 0.519*"bottles" + 0.287*"bottle" + -0.173*"bath" + 0.165*"nipples" + -0.122*"seat" + 0.108*"milk" + 0.098*"nipple" + -0.098*"gate" + 0.089*"pump"
Topic: 2 Word: 0.520*"tub" + -0.458*"gate" + -0.237*"seat" + 0.207*"bottles" + -0.186*"potty" + -0.182*"diaper" + 0.136*"water" + -0.135*"toilet" + 0.134*"bath" + -0.126*"diapers"
Topic: 3 Word: -0.748*"gate" + 0.355*"seat" + 0.245*"potty" + 0.190*"toilet" + 0.143*"diaper" + -0.124*"gates" + -0.120*"tub" + 0.098*"chair" + 0.096*"bags" + 0.093*"diapers"
Topic: 4 Word: 0.425*"seat" + -0.411*"diaper" + -0.307*"bags" + 0.278*"potty" + -0.267*"diapers" + -0.237*"pail" + -0.237*"bag" + 0.220*"toilet" + -0.152*"smell" + -0.144*"trash"
Topic: 5 Word: -0.528*"toy" + 0.311*"seat" + -0.227*"music" + 0.189*"diaper" + 0.186*"tub" + 0.184*"gate" + 0.168*

In [51]:
for index, score in sorted(lsi_model[bow_corpus[4310]],key=lambda tup:-1*tup[1]):
  print("\nScore:{}\t \nTopic:{}".format(score,lsi_model.print_topic(index,10)))


Score:0.36824137484569447	 
Topic:-0.553*"bottles" + 0.540*"tub" + 0.299*"baby" + -0.294*"bottle" + 0.133*"gate" + 0.125*"months" + 0.121*"bath" + 0.118*"seat" + 0.109*"son" + -0.106*"nipples"

Score:0.10022226064378975	 
Topic:0.477*"diaper" + -0.387*"seat" + -0.314*"baby" + 0.284*"bag" + 0.266*"bags" + 0.239*"diapers" + 0.188*"pail" + 0.166*"bottle" + 0.150*"time" + -0.147*"potty"

Score:-0.14149001672561734	 
Topic:0.713*"bottle" + -0.483*"bottles" + 0.284*"water" + -0.157*"diaper" + 0.148*"seat" + 0.134*"milk" + -0.107*"son" + -0.091*"months" + -0.084*"diapers" + -0.078*"bag"

Score:-0.307759267018422	 
Topic:-0.674*"gate" + 0.539*"tub" + -0.168*"seat" + 0.129*"water" + -0.117*"diaper" + -0.110*"product" + -0.110*"potty" + 0.103*"bottles" + 0.099*"bath" + -0.098*"one"

Score:-0.39105892160722516	 
Topic:0.628*"gate" + -0.418*"seat" + 0.330*"tub" + -0.242*"potty" + -0.180*"diaper" + -0.146*"son" + -0.146*"toilet" + -0.119*"product" + 0.113*"bottles" + -0.104*"bag"

Score:-0.5237749

In [52]:
unseen_document = 'That Pacifier is a very bad products'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lsi_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.13580898449777484	 Topic: 0.056*"baby" + 0.026*"bottles" + 0.022*"monitor" + 0.020*"bottle" + 0.020*"time"
Score: 0.1073368013835029	 Topic: 0.052*"toy" + 0.045*"son" + 0.028*"cups" + 0.018*"months" + 0.017*"baby"
Score: 0.07417207920927511	 Topic: 0.053*"baby" + 0.026*"one" + 0.026*"diaper" + 0.023*"product" + 0.022*"diapers"
Score: 0.06762209685231292	 Topic: 0.098*"bottles" + 0.035*"baby" + 0.024*"time" + 0.023*"nipples" + 0.021*"bottle"
Score: 0.02520239213924264	 Topic: 0.026*"stool" + 0.024*"daughter" + 0.020*"son" + 0.020*"bed" + 0.017*"product"
Score: -0.012778849332133697	 Topic: 0.081*"gate" + 0.048*"tub" + 0.031*"baby" + 0.019*"product" + 0.019*"time"
Score: -0.11040737721106256	 Topic: 0.052*"baby" + 0.038*"daughter" + 0.035*"time" + 0.034*"seat" + 0.024*"bottles"
Score: -0.11873621656973388	 Topic: 0.054*"baby" + 0.027*"months" + 0.025*"son" + 0.024*"diaper" + 0.024*"time"
Score: -0.12462823815517327	 Topic: 0.049*"gate" + 0.033*"baby" + 0.026*"product" + 0.025*"s